In [1]:
# ==================== INSTALLATION (Cell 1) ====================
!pip install -q openai-whisper pydub
!pip install -q pyannote.audio==3.1.1
!pip install -q numpy==1.26.4

print("✅ Installation complete! Run main code next.")

# ==================== MAIN CODE (Cell 2) ====================

import whisper
import torch
from pydub import AudioSegment
from pyannote.audio import Pipeline
from transformers import T5ForConditionalGeneration, T5Tokenizer
import os
import json
from datetime import timedelta
import warnings
warnings.filterwarnings('ignore')

print("📦 Libraries loaded successfully!")


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.2/803.2 kB 12.4 MB/s eta 0:00:00a 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 115.9 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 90.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 39.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 3.0 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 9.

/usr/local/lib/python3.11/dist-packages/pyannote/audio/core/io.py:43: UserWarning: torchaudio._backend.set_audio_backend has been deprecated. With dispatcher enabled, this function is no-op. You can remove the function call.
  torchaudio.set_audio_backend("soundfile")
2025-12-30 10:21:00.841418: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1767090061.207307      47 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1767090061.306148      47 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

📦 Libraries loaded successfully!


In [ ]:
# Complete Meeting Pipeline: Translation + Diarization + Summarization + BRD + PO
# Whisper Large → Pyannote 3.1 → FLAN-T5-Large
# Best quality at each stage

# ==================== CONFIGURATION ====================
class Config:
    # HuggingFace token for Pyannote
    HUGGINGFACE_TOKEN = "YOUR_HUGGINGFACE_TOKEN_HERE"  # Get from https://huggingface.co/settings/tokens
    
    # Audio file
    AUDIO_PATH = "/kaggle/input/eng-hinbi-marathi-mix-audio/Random recording for ReqGen.m4a"
    
    # Model settings
    WHISPER_MODEL = "large"  # Translation model
    SUMMARIZER_MODEL = "google/flan-t5-large"  # Summarization model
    
    # Speaker settings
    MIN_SPEAKERS = None  # Auto-detect
    MAX_SPEAKERS = None
    
    # Summarization settings
    SUMMARY_MAX_LENGTH = 1024  # Max length of summary
    SUMMARY_MIN_LENGTH = 256   # Min length of summary

config = Config()

print(f"⚙️  Configuration:")
print(f"   Translation: Whisper {config.WHISPER_MODEL}")
print(f"   Diarization: Pyannote 3.1 (DER: 11.2%)")
print(f"   Summarization: {config.SUMMARIZER_MODEL}")

# ==================== AUDIO PREPARATION ====================
def prepare_audio(audio_path):
    """Convert audio to WAV format"""
    print(f"\n📁 Loading audio: {audio_path}")
    
    audio = AudioSegment.from_file(audio_path)
    audio = audio.set_channels(1).set_frame_rate(16000)
    
    wav_path = "audio_processed.wav"
    audio.export(wav_path, format="wav")
    
    duration_minutes = len(audio) / 1000 / 60
    print(f"⏱️  Duration: {duration_minutes:.2f} minutes")
    
    return wav_path, duration_minutes

# ==================== STEP 1: TRANSLATION ====================
def translate_with_whisper(audio_path, model_size="large"):
    """Translate audio to English with Whisper"""
    print("\n" + "=" * 70)
    print("STEP 1: TRANSLATION WITH WHISPER LARGE")
    print("=" * 70)
    
    device = "cuda" if torch.cuda.is_available() else "cpu"
    print(f"💻 Device: {device}")
    
    print(f"🎤 Loading Whisper {model_size}...")
    model = whisper.load_model(model_size, device=device)
    
    print("⏳ Translating to English...")
    result = model.transcribe(
        audio_path,
        task="translate",
        word_timestamps=True,
        verbose=True
    )
    
    print(f"✅ Translation complete!")
    print(f"🌍 {result['language']} → English")
    print(f"📝 Segments: {len(result['segments'])}")
    
    return result

# ==================== STEP 2: DIARIZATION ====================
def diarize_with_pyannote(audio_path, hf_token, min_speakers=None, max_speakers=None):
    """Identify speakers with Pyannote 3.1"""
    print("\n" + "=" * 70)
    print("STEP 2: SPEAKER DIARIZATION (PYANNOTE 3.1)")
    print("=" * 70)
    
    if not hf_token or hf_token == "YOUR_HF_TOKEN":
        print("❌ ERROR: HuggingFace token required!")
        print("\nAccept terms for ALL models:")
        print("   • https://huggingface.co/pyannote/speaker-diarization-3.1")
        print("   • https://huggingface.co/pyannote/segmentation-3.0")
        print("   • https://huggingface.co/pyannote/embedding")
        return None
    
    try:
        print("🎯 Loading Pyannote 3.1...")
        pipeline = Pipeline.from_pretrained(
            "pyannote/speaker-diarization-3.1",
            use_auth_token=hf_token
        )
        
        if torch.cuda.is_available():
            pipeline.to(torch.device("cuda"))
        
        print("⏳ Analyzing speakers...")
        diarization = pipeline(
            audio_path,
            min_speakers=min_speakers,
            max_speakers=max_speakers
        )
        
        speaker_segments = []
        for turn, _, speaker in diarization.itertracks(yield_label=True):
            speaker_segments.append({
                "start": turn.start,
                "end": turn.end,
                "speaker": speaker
            })
        
        num_speakers = len(set([s["speaker"] for s in speaker_segments]))
        print(f"✅ Detected {num_speakers} speakers")
        
        return speaker_segments
        
    except Exception as e:
        print(f"❌ Error: {e}")
        return None

# ==================== STEP 3: MERGE ====================
def merge_translation_with_speakers(whisper_result, pyannote_segments):
    """Combine translation with speaker labels"""
    print("\n" + "=" * 70)
    print("STEP 3: MERGING TRANSLATION WITH SPEAKERS")
    print("=" * 70)
    
    if not pyannote_segments:
        print("❌ No speaker segments")
        return None
    
    whisper_segments = whisper_result['segments']
    merged_segments = []
    
    for w_seg in whisper_segments:
        seg_start = w_seg['start']
        seg_end = w_seg['end']
        
        assigned_speaker = "UNKNOWN"
        max_overlap = 0
        
        for p_seg in pyannote_segments:
            overlap_start = max(seg_start, p_seg['start'])
            overlap_end = min(seg_end, p_seg['end'])
            overlap = max(0, overlap_end - overlap_start)
            
            if overlap > max_overlap:
                max_overlap = overlap
                assigned_speaker = p_seg['speaker']
        
        merged_segments.append({
            'start': seg_start,
            'end': seg_end,
            'text': w_seg['text'].strip(),
            'speaker': assigned_speaker
        })
    
    # Rename speakers
    speaker_counts = {}
    for seg in merged_segments:
        speaker = seg['speaker']
        speaker_counts[speaker] = speaker_counts.get(speaker, 0) + 1
    
    sorted_speakers = sorted(speaker_counts.items(), key=lambda x: x[1], reverse=True)
    speaker_mapping = {}
    for i, (original_label, _) in enumerate(sorted_speakers, 1):
        speaker_mapping[original_label] = f"Speaker {i}"
    
    for seg in merged_segments:
        seg['speaker'] = speaker_mapping.get(seg['speaker'], seg['speaker'])
    
    print(f"✅ Merged {len(merged_segments)} segments")
    return merged_segments

# ==================== STEP 4: FORMAT ====================
def format_transcript(merged_segments):
    """Create readable transcript"""
    lines_with_timestamps = []
    lines_plain = []
    
    current_speaker = None
    current_texts = []
    segment_start_time = None
    
    for seg in merged_segments:
        if seg['speaker'] != current_speaker:
            if current_speaker:
                timestamp = str(timedelta(seconds=int(segment_start_time)))
                lines_with_timestamps.append(
                    f"[{timestamp}] {current_speaker}: {' '.join(current_texts)}"
                )
            
            current_speaker = seg['speaker']
            current_texts = [seg['text']]
            segment_start_time = seg['start']
        else:
            current_texts.append(seg['text'])
    
    if current_speaker:
        timestamp = str(timedelta(seconds=int(segment_start_time)))
        lines_with_timestamps.append(
            f"[{timestamp}] {current_speaker}: {' '.join(current_texts)}"
        )
    
    transcript_with_timestamps = "\n\n".join(lines_with_timestamps)
    
    # Plain format
    current_speaker = None
    current_texts = []
    
    for seg in merged_segments:
        if seg['speaker'] != current_speaker:
            if current_speaker:
                lines_plain.append(f"{current_speaker}: {' '.join(current_texts)}")
            current_speaker = seg['speaker']
            current_texts = [seg['text']]
        else:
            current_texts.append(seg['text'])
    
    if current_speaker:
        lines_plain.append(f"{current_speaker}: {' '.join(current_texts)}")
    
    transcript_plain = "\n\n".join(lines_plain)
    
    return transcript_with_timestamps, transcript_plain

# ==================== STEP 5: SUMMARIZATION WITH FLAN-T5 ====================
def summarize_with_flan_t5(transcript, duration_minutes, num_speakers, model_name="google/flan-t5-large"):
    """
    Generate comprehensive summary using FLAN-T5-Large
    FLAN-T5 is instruction-tuned for better summarization quality
    """
    print("\n" + "=" * 70)
    print("STEP 5: SUMMARIZATION WITH FLAN-T5-LARGE")
    print("=" * 70)
    
    device = "cuda" if torch.cuda.is_available() else "cpu"
    print(f"💻 Device: {device}")
    
    print(f"🤖 Loading {model_name}...")
    print("   (This may take a few minutes...)")
    
    try:
        tokenizer = T5Tokenizer.from_pretrained(model_name)
        model = T5ForConditionalGeneration.from_pretrained(model_name)
        model = model.to(device)
        model.eval()
        
        print("✅ Model loaded successfully!")
        
        # Determine summary approach based on meeting length
        if duration_minutes < 10:
            summary_type = "brief"
            max_length = 256
            min_length = 128
        elif duration_minutes < 30:
            summary_type = "moderate"
            max_length = 512
            min_length = 256
        elif duration_minutes < 60:
            summary_type = "detailed"
            max_length = 768
            min_length = 384
        else:
            summary_type = "comprehensive"
            max_length = 1024
            min_length = 512
        
        print(f"📊 Meeting: {duration_minutes:.1f} min → {summary_type} summary")
        print(f"   Target length: {min_length}-{max_length} tokens")
        
        # Create instruction-based prompt for FLAN-T5
        instruction = f"""Analyze this {duration_minutes:.1f}-minute meeting transcript with {num_speakers} speakers and provide a {summary_type} summary.

Your summary must include:
1. Executive Summary: Brief overview of meeting purpose and outcome
2. Key Discussion Points: Main topics discussed by each speaker
3. Speaker Contributions: Notable points from each participant
4. Decisions Made: Any decisions or conclusions reached
5. Action Items: Tasks assigned to specific people
6. Next Steps: Follow-up actions or future meetings

Meeting Transcript:
{transcript}

Provide a well-structured, {summary_type} summary:"""

        print("⏳ Generating summary...")
        print("   This may take 2-5 minutes depending on transcript length...")
        
        # Tokenize input
        inputs = tokenizer(
            instruction,
            return_tensors="pt",
            max_length=2048,  # Input limit
            truncation=True
        ).to(device)
        
        # Generate summary
        with torch.no_grad():
            outputs = model.generate(
                inputs.input_ids,
                max_length=max_length,
                min_length=min_length,
                num_beams=4,  # Beam search for better quality
                length_penalty=2.0,  # Encourage longer summaries
                early_stopping=True,
                no_repeat_ngram_size=3,  # Avoid repetition
                temperature=0.7,  # Some creativity
                do_sample=False  # Deterministic for consistency
            )
        
        # Decode summary
        summary = tokenizer.decode(outputs[0], skip_special_tokens=True)
        
        print("✅ Summary generated!")
        print(f"📝 Summary length: {len(summary.split())} words")
        
        # Format the summary with sections
        formatted_summary = f"""# MEETING SUMMARY
**Duration**: {duration_minutes:.1f} minutes
**Speakers**: {num_speakers}
**Generated by**: FLAN-T5-Large

---

{summary}

---
*Note: This summary was automatically generated using FLAN-T5-Large, a state-of-the-art instruction-tuned language model.*
"""
        
        return formatted_summary
        
    except Exception as e:
        print(f"❌ Summarization error: {e}")
        return f"Error generating summary: {str(e)}"

# ==================== STEP 6: GENERATE BRD & PURCHASE ORDERS ====================
def generate_brd_from_summary(summary, transcript, duration_minutes, num_speakers):
    """
    Generate Business Requirements Document from meeting summary
    """
    print("\n" + "=" * 70)
    print("STEP 6A: GENERATING BUSINESS REQUIREMENTS DOCUMENT")
    print("=" * 70)
    
    device = "cuda" if torch.cuda.is_available() else "cpu"
    
    try:
        print("🤖 Loading FLAN-T5 for BRD generation...")
        tokenizer = T5Tokenizer.from_pretrained(config.SUMMARIZER_MODEL)
        model = T5ForConditionalGeneration.from_pretrained(config.SUMMARIZER_MODEL)
        model = model.to(device)
        model.eval()
        
        # Create BRD-specific prompt
        brd_prompt = f"""You are a business analyst. Based on this meeting summary and transcript, create a professional Business Requirements Document (BRD).

Meeting Summary:
{summary}

Meeting Transcript:
{transcript[:2000]}

Generate a structured BRD with these sections:
1. DOCUMENT CONTROL (version, date, status)
2. EXECUTIVE SUMMARY
3. BUSINESS OBJECTIVES
4. SCOPE (in-scope and out-of-scope items)
5. STAKEHOLDERS AND ROLES
6. FUNCTIONAL REQUIREMENTS (numbered list)
7. NON-FUNCTIONAL REQUIREMENTS
8. CONSTRAINTS AND ASSUMPTIONS
9. SUCCESS CRITERIA
10. RISKS AND DEPENDENCIES
11. TIMELINE AND MILESTONES

Create a detailed, professional BRD:"""

        print("⏳ Generating BRD... (this may take 2-3 minutes)")
        
        inputs = tokenizer(
            brd_prompt,
            return_tensors="pt",
            max_length=2048,
            truncation=True
        ).to(device)
        
        with torch.no_grad():
            outputs = model.generate(
                inputs.input_ids,
                max_length=1024,
                min_length=512,
                num_beams=4,
                length_penalty=2.0,
                early_stopping=True,
                no_repeat_ngram_size=3,
                temperature=0.7
            )
        
        brd_content = tokenizer.decode(outputs[0], skip_special_tokens=True)
        
        # Format BRD with proper structure
        from datetime import datetime
        today = datetime.now().strftime("%Y-%m-%d")
        
        formatted_brd = f"""# BUSINESS REQUIREMENTS DOCUMENT (BRD)

---

## DOCUMENT CONTROL

| Field | Value |
|-------|-------|
| **Document Title** | Business Requirements Document - Meeting Analysis |
| **Version** | 1.0 |
| **Date** | {today} |
| **Status** | Draft |
| **Meeting Duration** | {duration_minutes:.1f} minutes |
| **Participants** | {num_speakers} speakers |
| **Generated By** | FLAN-T5-Large AI System |

---

## DOCUMENT PURPOSE

This Business Requirements Document captures the business needs, objectives, and requirements discussed in the meeting held on {today}. It serves as a formal record of what was agreed upon and what needs to be delivered.

---

{brd_content}

---

## APPROVAL SIGNATURES

| Role | Name | Signature | Date |
|------|------|-----------|------|
| **Business Sponsor** | _______________ | _______________ | ______ |
| **Project Manager** | _______________ | _______________ | ______ |
| **Technical Lead** | _______________ | _______________ | ______ |
| **Business Analyst** | _______________ | _______________ | ______ |

---

## REVISION HISTORY

| Version | Date | Author | Description |
|---------|------|--------|-------------|
| 1.0 | {today} | AI System | Initial draft generated from meeting |

---

*This document was automatically generated using AI analysis of meeting transcript. Please review and validate all information before formal approval.*
"""
        
        print("✅ BRD generated successfully!")
        return formatted_brd
        
    except Exception as e:
        print(f"❌ BRD generation error: {e}")
        return None

def generate_purchase_orders(summary, transcript):
    """
    Generate Purchase Orders from meeting discussion
    """
    print("\n" + "=" * 70)
    print("STEP 6B: GENERATING PURCHASE ORDERS")
    print("=" * 70)
    
    device = "cuda" if torch.cuda.is_available() else "cpu"
    
    try:
        print("🤖 Loading FLAN-T5 for PO generation...")
        tokenizer = T5Tokenizer.from_pretrained(config.SUMMARIZER_MODEL)
        model = T5ForConditionalGeneration.from_pretrained(config.SUMMARIZER_MODEL)
        model = model.to(device)
        model.eval()
        
        # Create PO extraction prompt
        po_prompt = f"""You are a procurement specialist. Analyze this meeting summary and transcript to identify any items, services, or resources that need to be purchased.

Meeting Summary:
{summary}

Meeting Transcript:
{transcript[:2000]}

Extract and list:
1. Items/services to be purchased
2. Quantities (if mentioned)
3. Estimated costs/budgets (if mentioned)
4. Vendors/suppliers (if mentioned)
5. Required delivery dates (if mentioned)
6. Purpose/justification for each purchase
7. Department/cost center responsible

Format as a structured list of purchase requirements:"""

        print("⏳ Extracting purchase requirements...")
        
        inputs = tokenizer(
            po_prompt,
            return_tensors="pt",
            max_length=2048,
            truncation=True
        ).to(device)
        
        with torch.no_grad():
            outputs = model.generate(
                inputs.input_ids,
                max_length=768,
                min_length=256,
                num_beams=4,
                length_penalty=2.0,
                early_stopping=True,
                no_repeat_ngram_size=3
            )
        
        po_content = tokenizer.decode(outputs[0], skip_special_tokens=True)
        
        # Format as professional PO document
        from datetime import datetime, timedelta
        today = datetime.now().strftime("%Y-%m-%d")
        po_number = f"PO-{datetime.now().strftime('%Y%m%d')}-001"
        delivery_date = (datetime.now() + timedelta(days=30)).strftime("%Y-%m-%d")
        
        formatted_po = f"""# PURCHASE ORDER REQUEST

---

## PURCHASE ORDER DETAILS

| Field | Value |
|-------|-------|
| **PO Number** | {po_number} |
| **Date Issued** | {today} |
| **Requested By** | [To be filled] |
| **Department** | [To be filled] |
| **Approval Status** | Pending Approval |
| **Delivery Date Requested** | {delivery_date} |

---

## VENDOR INFORMATION

| Field | Value |
|-------|-------|
| **Vendor Name** | [To be determined based on requirements] |
| **Vendor Contact** | [To be filled] |
| **Payment Terms** | Net 30 |
| **Shipping Method** | [To be determined] |

---

## ITEMS/SERVICES REQUESTED

{po_content}

---

## COST SUMMARY

| Category | Estimated Amount |
|----------|-----------------|
| **Subtotal** | $[To be calculated] |
| **Tax (if applicable)** | $[To be calculated] |
| **Shipping** | $[To be calculated] |
| **Total** | $[To be calculated] |

---

## BUSINESS JUSTIFICATION

Based on the meeting discussion, these purchases are required to support:
- Project objectives and deliverables
- Timeline commitments
- Resource needs identified by team
- Strategic initiatives discussed

---

## BUDGET ALLOCATION

| Cost Center | Budget Code | Amount |
|-------------|-------------|--------|
| [Department] | [Code] | $[Amount] |

---

## APPROVAL WORKFLOW

| Level | Approver | Signature | Date | Status |
|-------|----------|-----------|------|--------|
| **Requestor** | [Name] | _______________ | ______ | Pending |
| **Manager** | [Name] | _______________ | ______ | Pending |
| **Finance** | [Name] | _______________ | ______ | Pending |
| **Procurement** | [Name] | _______________ | ______ | Pending |

---

## TERMS AND CONDITIONS

1. All items must meet specified quality standards
2. Delivery must be completed by requested date
3. Invoices should reference this PO number
4. Any changes to this order must be approved in writing
5. Payment will be processed upon receipt and verification of goods/services

---

## ADDITIONAL NOTES

*This purchase order was automatically generated from meeting analysis. Please review all requirements carefully and fill in missing information before submitting for approval.*

**Generated from meeting held on:** {today}
**Source:** AI-analyzed meeting transcript

---

## ATTACHMENTS

- Meeting transcript
- Meeting summary
- Business Requirements Document (if applicable)
- Technical specifications (if applicable)

---

*For questions regarding this purchase order, please contact the requesting department or procurement team.*
"""
        
        print("✅ Purchase Order generated successfully!")
        return formatted_po
        
    except Exception as e:
        print(f"❌ PO generation error: {e}")
        return None
def complete_meeting_pipeline(
    audio_path,
    whisper_model="large",
    hf_token=None,
    summarizer_model="google/flan-t5-large",
    min_speakers=None,
    max_speakers=None
):
    """
    Complete pipeline with best-in-class models at each stage
    """
    print("=" * 70)
    print("🎙️  COMPLETE MEETING ANALYSIS PIPELINE")
    print("=" * 70)
    print("\n🏆 Best-in-class models:")
    print("   1. Whisper Large - Translation")
    print("   2. Pyannote 3.1 - Speaker Diarization (11.2% DER)")
    print("   3. FLAN-T5-Large - Comprehensive Summarization")
    print("\n📋 Pipeline stages:")
    print("   Stage 1: Translate audio to English")
    print("   Stage 2: Identify speakers with voice embeddings")
    print("   Stage 3: Merge translation with speaker labels")
    print("   Stage 4: Format readable transcript")
    print("   Stage 5: Generate comprehensive summary")
    print("=" * 70)
    
    # Prepare audio
    wav_path, duration = prepare_audio(audio_path)
    
    # Stage 1: Translate
    translation = translate_with_whisper(wav_path, whisper_model)
    
    # Stage 2: Diarize
    speaker_segments = diarize_with_pyannote(
        wav_path,
        hf_token,
        min_speakers,
        max_speakers
    )
    
    if not speaker_segments:
        print("\n❌ Pipeline failed at diarization stage")
        return None
    
    # Stage 3: Merge
    merged_segments = merge_translation_with_speakers(translation, speaker_segments)
    
    if not merged_segments:
        print("\n❌ Pipeline failed at merging stage")
        return None
    
    # Stage 4: Format
    transcript_ts, transcript_plain = format_transcript(merged_segments)
    
    num_speakers = len(set([s['speaker'] for s in merged_segments]))
    
    print("\n" + "=" * 70)
    print("📊 TRANSCRIPT GENERATED")
    print("=" * 70)
    print(f"🌍 Language: {translation['language']} → English")
    print(f"👥 Speakers: {num_speakers}")
    print(f"⏱️  Duration: {duration:.1f} minutes")
    print(f"📝 Segments: {len(merged_segments)}")
    
    # Display speaker stats
    speaker_counts = {}
    for seg in merged_segments:
        speaker = seg['speaker']
        speaker_counts[speaker] = speaker_counts.get(speaker, 0) + 1
    
    print(f"\n👥 Speaker participation:")
    for speaker in sorted(speaker_counts.keys(), key=lambda x: int(x.split()[1])):
        count = speaker_counts[speaker]
        percentage = (count / len(merged_segments)) * 100
        print(f"   {speaker}: {count} segments ({percentage:.1f}%)")
    
    # Display transcript preview
    print("\n" + "-" * 70)
    print("TRANSCRIPT PREVIEW:")
    print("-" * 70)
    preview = transcript_ts[:1000]
    print(preview + ("..." if len(transcript_ts) > 1000 else ""))
    
    # Stage 5: Summarize
    summary = summarize_with_flan_t5(
        transcript_plain,
        duration,
        num_speakers,
        config.SUMMARIZER_MODEL
    )
    
    print("\n" + "=" * 70)
    print("📋 SUMMARY GENERATED")
    print("=" * 70)
    print(summary[:500] + ("..." if len(summary) > 500 else ""))
    
    # Stage 6A: Generate BRD
    print("\n" + "=" * 70)
    print("📄 GENERATING BUSINESS DOCUMENTS")
    print("=" * 70)
    
    brd = generate_brd_from_summary(
        summary,
        transcript_plain,
        duration,
        num_speakers
    )
    
    # Stage 6B: Generate Purchase Orders
    purchase_order = generate_purchase_orders(
        summary,
        transcript_plain
    )
    
    # Save results
    print("\n" + "=" * 70)
    print("💾 SAVING FILES")
    print("=" * 70)
    
    output = {
        "original_language": translation['language'],
        "translated_to": "English",
        "duration_minutes": duration,
        "num_speakers": num_speakers,
        "models_used": {
            "translation": f"Whisper {whisper_model}",
            "diarization": "Pyannote 3.1 (DER: 11.2%)",
            "summarization": config.SUMMARIZER_MODEL
        },
        "transcript_with_timestamps": transcript_ts,
        "transcript_plain": transcript_plain,
        "summary": summary,
        "brd": brd,
        "purchase_order": purchase_order,
        "segments": merged_segments
    }
    
    with open("transcript_with_speakers.txt", "w", encoding="utf-8") as f:
        f.write(transcript_plain)
    print("✅ transcript_with_speakers.txt")
    
    with open("transcript_timestamps.txt", "w", encoding="utf-8") as f:
        f.write(transcript_ts)
    print("✅ transcript_timestamps.txt")
    
    with open("meeting_summary.md", "w", encoding="utf-8") as f:
        f.write(summary)
    print("✅ meeting_summary.md")
    
    if brd:
        with open("business_requirements_document.md", "w", encoding="utf-8") as f:
            f.write(brd)
        print("✅ business_requirements_document.md")
    
    if purchase_order:
        with open("purchase_order_request.md", "w", encoding="utf-8") as f:
            f.write(purchase_order)
        print("✅ purchase_order_request.md")
    
    with open("complete_output.json", "w", encoding="utf-8") as f:
        json.dump(output, f, indent=2, ensure_ascii=False)
    print("✅ complete_output.json")
    
    print("\n" + "=" * 70)
    print("🎉 PIPELINE COMPLETE!")
    print("=" * 70)
    print(f"✅ Translation: {translation['language']} → English")
    print(f"✅ Speakers identified: {num_speakers}")
    print(f"✅ Comprehensive summary generated")
    print(f"✅ Business Requirements Document created")
    print(f"✅ Purchase Order template generated")
    print(f"✅ All files saved successfully")
    print("\n📁 Generated Documents:")
    print(f"   1. transcript_with_speakers.txt - Clean transcript")
    print(f"   2. transcript_timestamps.txt - With timestamps")
    print(f"   3. meeting_summary.md - AI summary")
    print(f"   4. business_requirements_document.md - BRD ⭐")
    print(f"   5. purchase_order_request.md - PO template ⭐")
    print(f"   6. complete_output.json - All data")
    
    # Cleanup
    if os.path.exists(wav_path):
        os.remove(wav_path)
    
    return output

# ==================== USAGE FUNCTIONS ====================

def run_complete_pipeline():
    """Run full pipeline with all stages"""
    from kaggle_secrets import UserSecretsClient
    secrets = UserSecretsClient()
    
    try:
        hf_token = secrets.get_secret("HUGGINGFACE_TOKEN")
    except:
        print("❌ HUGGINGFACE_TOKEN not found in Kaggle Secrets!")
        return None
    
    return complete_meeting_pipeline(
        audio_path=config.AUDIO_PATH,
        whisper_model=config.WHISPER_MODEL,
        hf_token=hf_token,
        summarizer_model=config.SUMMARIZER_MODEL,
        min_speakers=config.MIN_SPEAKERS,
        max_speakers=config.MAX_SPEAKERS
    )

def run_with_medium_models():
    """Faster processing with medium models"""
    from kaggle_secrets import UserSecretsClient
    secrets = UserSecretsClient()
    hf_token = secrets.get_secret("HUGGINGFACE_TOKEN")
    
    return complete_meeting_pipeline(
        audio_path=config.AUDIO_PATH,
        whisper_model="medium",  # Faster
        hf_token=hf_token,
        summarizer_model="google/flan-t5-base",  # Smaller, faster
        min_speakers=config.MIN_SPEAKERS,
        max_speakers=config.MAX_SPEAKERS
    )

def run_with_known_speakers(num_speakers):
    """When you know exact speaker count"""
    from kaggle_secrets import UserSecretsClient
    secrets = UserSecretsClient()
    hf_token = secrets.get_secret("HUGGINGFACE_TOKEN")
    
    return complete_meeting_pipeline(
        audio_path=config.AUDIO_PATH,
        whisper_model=config.WHISPER_MODEL,
        hf_token=hf_token,
        summarizer_model=config.SUMMARIZER_MODEL,
        min_speakers=num_speakers,
        max_speakers=num_speakers
    )

# ==================== MAIN EXECUTION ====================
if __name__ == "__main__":
    print("\n🚀 Starting complete meeting analysis pipeline...\n")
    
    # Choose ONE:
    
    # Option 1: Full pipeline (RECOMMENDED)
    result = run_complete_pipeline()
    
    # Option 2: Faster with medium models
    # result = run_with_medium_models()
    
    # Option 3: Known speaker count
    # result = run_with_known_speakers(num_speakers=3)
    
    if result:
        print(f"\n✅ SUCCESS!")
        print(f"   Original: {result['original_language']}")
        print(f"   Speakers: {result['num_speakers']}")
        print(f"   Duration: {result['duration_minutes']:.1f} min")
        print(f"\n📁 Check these files:")
        print(f"   • transcript_with_speakers.txt")
        print(f"   • transcript_timestamps.txt")
        print(f"   • meeting_summary.md")
        print(f"   • business_requirements_document.md ⭐ NEW!")
        print(f"   • purchase_order_request.md ⭐ NEW!")
        print(f"   • complete_output.json")
    else:
        print("\n❌ Pipeline failed. Check errors above.")

⚙️  Configuration:
   Translation: Whisper large
   Diarization: Pyannote 3.1 (DER: 11.2%)
   Summarization: google/flan-t5-large

🚀 Starting complete meeting analysis pipeline...

🎙️  COMPLETE MEETING ANALYSIS PIPELINE

🏆 Best-in-class models:
   1. Whisper Large - Translation
   2. Pyannote 3.1 - Speaker Diarization (11.2% DER)
   3. FLAN-T5-Large - Comprehensive Summarization

📋 Pipeline stages:
   Stage 1: Translate audio to English
   Stage 2: Identify speakers with voice embeddings
   Stage 3: Merge translation with speaker labels
   Stage 4: Format readable transcript
   Stage 5: Generate comprehensive summary

📁 Loading audio: /kaggle/input/eng-hinbi-marathi-mix-audio/Random recording for ReqGen.m4a
⏱️  Duration: 2.60 minutes

STEP 1: TRANSLATION WITH WHISPER LARGE
💻 Device: cuda
🎤 Loading Whisper large...


100%|██████████████████████████████████████| 2.88G/2.88G [00:13<00:00, 223MiB/s]


⏳ Translating to English...
Detecting language using up to the first 30 seconds. Use `--language` to specify the language
Detected language: English
[00:00.000 --> 00:13.840]  Hello everyone, good morning. I hope I am audible to everyone. Today's topic of our discussion will be on business card detection.
[00:13.840 --> 00:26.400]  Right. So the agenda in discussing this project is that we have to detect the business card from a given video.
[00:28.360 --> 00:33.620]  Which means that in that video, we will be showing the business card.
[00:34.040 --> 00:40.400]  The person who has that business card will be showing it in the video, you will see that.
[00:40.460 --> 00:41.100]  And then,
[00:41.580 --> 00:41.900]  and then,
[00:42.640 --> 00:45.640]  I will be showing you the details in detail.
[00:45.640 --> 00:50.620]  So what this means is that we will be showing all the details on that business card,
[00:50.900 --> 00:58.040]  like the name of the card holder or the name of the bus

config.yaml:   0%|          | 0.00/469 [00:00<?, ?B/s]

DEBUG:speechbrain.utils.checkpoints:Registered checkpoint save hook for _speechbrain_save
DEBUG:speechbrain.utils.checkpoints:Registered checkpoint load hook for _speechbrain_load
DEBUG:speechbrain.utils.checkpoints:Registered checkpoint save hook for save
DEBUG:speechbrain.utils.checkpoints:Registered checkpoint load hook for load
DEBUG:speechbrain.utils.checkpoints:Registered checkpoint save hook for _save
DEBUG:speechbrain.utils.checkpoints:Registered checkpoint load hook for _recover


...

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


✅ Model loaded successfully!
📊 Meeting: 2.6 min → brief summary
   Target length: 128-256 tokens
⏳ Generating summary...
   This may take 2-5 minutes depending on transcript length...
✅ Summary generated!
📝 Summary length: 117 words

📋 SUMMARY GENERATED
# MEETING SUMMARY
**Duration**: 2.6 minutes
**Speakers**: 1
**Generated by**: FLAN-T5-Large

---

Today's topic of the meeting is business card detection. Speakers discussed how to extract business card details from a video and how to incorporate it into a system or in a business. Speaker 1: Hello everyone, good morning. I hope I am audible to everyone. Today'd topic of our discussion will be on business card recognition. Speaker 2: Today, we are going to discuss how to detect the business card ...

📄 GENERATING BUSINESS DOCUMENTS

STEP 6A: GENERATING BUSINESS REQUIREMENTS DOCUMENT
🤖 Loading FLAN-T5 for BRD generation...


The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


⏳ Generating BRD... (this may take 2-3 minutes)
✅ BRD generated successfully!

STEP 6B: GENERATING PURCHASE ORDERS
🤖 Loading FLAN-T5 for PO generation...
⏳ Extracting purchase requirements...
✅ Purchase Order generated successfully!

💾 SAVING FILES
✅ transcript_with_speakers.txt
✅ transcript_timestamps.txt
✅ meeting_summary.md
✅ business_requirements_document.md
✅ purchase_order_request.md
✅ complete_output.json

🎉 PIPELINE COMPLETE!
✅ Translation: en → English
✅ Speakers identified: 1
✅ Comprehensive summary generated
✅ Business Requirements Document created
✅ Purchase Order template generated
✅ All files saved successfully

📁 Generated Documents:
   1. transcript_with_speakers.txt - Clean transcript
   2. transcript_timestamps.txt - With timestamps
   3. meeting_summary.md - AI summary
   4. business_requirements_document.md - BRD ⭐
   5. purchase_order_request.md - PO template ⭐
   6. complete_output.json - All data

✅ SUCCESS!
   Original: en
   Speakers: 1
   Duration: 2.6 min

📁 